#Extract fasta from gff

Notebook using Biopython and GFF to create a multi fasta file corresponding to the sequences indicated in a gff file and a reference fasta file

Define a gff representation and parsing class due to the unnecessary complexity of existing python parser

In [1]:
from gzip import open as gopen
from urllib import unquote
from collections import OrderedDict

class GffFeature(object):
    """Object representing a gff feature"""
    def __init__(self, seq_id, source, type, start, end, score, strand, phase, attributes):
        self.seq_id = seq_id 
        self.source = source
        self.type = type
        self.start = start 
        self.end = end
        self.score = score
        self.strand = strand
        self.phase = phase
        self.attributes = attributes
    
    def __str__ (self):
        """Return a gff formated line"""
        return "{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}".format(
            self.seq_id,
            self.source,
            self.type,
            self.start, 
            self.end,
            self.score,
            self.strand,
            self.phase,
            ";".join(["=".join(attribute) for attribute in self.attributes.items()]))     
    
class GffSequence(object):
    """Object representing a sequence in a gff file"""
    
    def __init__(self, seq_id, start, end):
        self.seq_id = seq_id
        self.start = start
        self.end = end
        self.features = []
        self.features_count = 0
    
    def __str__ (self):
        """Return a gff formated line"""
        return "##sequence-region {} {} {}\n{}".format(
            self.seq_id,
            self.start,
            self.end,
            "\n".join([str(feature) for feature in self.features]))
        
    def add_feature(self, source, type, start, end, score, strand, phase, attributes):
        self.features.append(GffFeature(self.seq_id, source, type, start, end, score, strand, phase, attributes))
        self.features_count += 1
        
class GffParser(object):
    """
    A minimalistic GFF3 format parser supporting gzip decompression.
    """

    ##### FONDAMENTAL METHODS #####
    
    def __init__(self, gff_file, feature_types=[]):
        """
        gff_file    Path to a standard gff3 file
        feature_types    restrict to the type of features indicated in the list (Default = all types) 
        """
        openFunc = gopen if gff_file.endswith(".gz") else open
        self.gff_dict = OrderedDict()
        self.gff_header = ""
        self.valid_features = 0
        self.all_features = 0
               
        with openFunc(gff_file) as gff:
            for line in gff:
                line = line.lstrip()
                if not line: continue
                
                # Parse sequence delimiter in gff file (ex: ##sequence-region chr1 1 248956422)
                if line.startswith("##sequence-region"):

                    # Part and verify that the number of fields is standard
                    parts = line.strip().split()
                    assert len(parts) == 4, "File format is not standard-compatible: Invalid sequence-region pragma"

                    # Create a GffSequence
                    seq_id = unquote(parts[1])
                    self.gff_dict[seq_id]= GffSequence (seq_id = seq_id, start = int(parts[2]), end = int(parts[3]))

                # Parse the general gff header
                elif line.startswith("#"):
                    self.gff_header+=line

                # Parse gff features
                else:

                    # Count all features
                    self.all_features += 1
                    if self.all_features % 100000 == 0:
                        print ("  {} features parsed".format(self.all_features))

                    # Part and verify that the number of fields is standard
                    parts = line.strip().split("\t")
                    assert len(parts) == 9, "File format is not standard-compatible : Invalid feature description line"

                    # Verify first if the feature type is allowed
                    type = '.' if parts[2] == "." else unquote(parts[2])
                    if not feature_types or type in feature_types:

                        # Count valid features
                        self.valid_features += 1

                        # Extract the sequence id and verify that the corresponding sequence-region pragma is already in the dict
                        seq_id = '.' if parts[0] == "." else unquote(parts[0])
                        assert seq_id != ".", "File format is not standard-compatible : Missing seq_id of a feature"
                        assert seq_id in self.gff_dict, "File format is not standard-compatible : Missing or misplaced sequence-region pragma"

                        # Finally add the feature to the feature list corresponding to the seqid entry in gff_dict
                        self.gff_dict[seq_id].add_feature(
                            source = '.' if parts[1] == "." else unquote(parts[1]),
                            type = type,
                            start = '.' if parts[3] == '.' else int(parts[3]),
                            end = '.' if parts[4] == '.' else int(parts[4]),
                            score = '.' if parts[5] == '.' else float(parts[4]),
                            strand = '.' if parts[6] == '.' else unquote(parts[6]),
                            phase = '.' if parts[7] == '.' else int(parts[7]),
                            attributes = self._parse_attributes(parts[8]))
    
    def __str__ (self):
        """Return a gff formated line"""
        
        return "{}{}".format(
            self.gff_header,
            "\n".join([str(sequence) for sequence in self.gff_dict.values()]))
    
    ##### PRIVATE METHODS #####
                                 
    def _parse_attributes(self, attribute_string):
        """Parse the GFF3 attribute column and return a dict"""
        
        # Empty dict to collect features attributes
        attributes_dict = OrderedDict()
        
        # Parse and return a dict of all attributes if attribute_string is defined    
        if attribute_string != ".":
            for attribute in attribute_string.split(";"):
                key, value = attribute.split("=")
                attributes_dict[unquote(key)] = unquote(value)
        return attributes_dict
                

Test GffFeature

In [2]:
gff_line = "chr1	HAVANA	gene	11869	14409	.	+	.	ID=ENSG00000223972.5;gene_id=ENSG00000223972.5;gene_type=transcribed_unprocessed_pseudogene;gene_status=KNOWN;gene_name=DDX11L1;level=2;havana_gene=OTTHUMG00000000961.2"
parts = gff_line.strip().split("\t")

attributes_dict = OrderedDict()
for attribute in parts[8].split(";"):
    key, value = attribute.split("=")
    attributes_dict[unquote(key)] = unquote(value)

gff_feature = GffFeature (parts[0], parts[1], parts[2], parts[3], parts[4], parts[5], parts[6], parts[7], attributes_dict)

print (gff_line)
print (gff_feature)

chr1	HAVANA	gene	11869	14409	.	+	.	ID=ENSG00000223972.5;gene_id=ENSG00000223972.5;gene_type=transcribed_unprocessed_pseudogene;gene_status=KNOWN;gene_name=DDX11L1;level=2;havana_gene=OTTHUMG00000000961.2
chr1	HAVANA	gene	11869	14409	.	+	.	ID=ENSG00000223972.5;gene_id=ENSG00000223972.5;gene_type=transcribed_unprocessed_pseudogene;gene_status=KNOWN;gene_name=DDX11L1;level=2;havana_gene=OTTHUMG00000000961.2


Test GffParser by parsing and rewriting the gff file afterward. OK with a simple file

In [111]:
gff = GffParser (gff_file="./test/sample.gff")
print (gff)

##gff-version 3
#description: -
#provider: adrien leger
#contact: adrien.leger <at> gmail <dot> com
#format: gff3
#date: 2015-08-03
##sequence-region chr1 1 1000
chr1	HAVANA	exon	200	300	.	+	.	ID=exon:ENST00000456328.2:1;Parent=ENST00000456328.2;gene_id=ENSG00000223972.5;transcript_id=ENST00000456328.2;gene_type=transcribed_unprocessed_pseudogene;gene_status=KNOWN;gene_name=DDX11L1;transcript_type=processed_transcript;transcript_status=KNOWN;transcript_name=DDX11L1-002;exon_number=1;exon_id=ENSE00002234944.1;level=2;transcript_support_level=1;havana_gene=OTTHUMG00000000961.2;havana_transcript=OTTHUMT00000362751.1;tag=basic
chr1	HAVANA	exon	600	700	.	+	.	ID=exon:ENST00000456328.2:2;Parent=ENST00000456328.2;gene_id=ENSG00000223972.5;transcript_id=ENST00000456328.2;gene_type=transcribed_unprocessed_pseudogene;gene_status=KNOWN;gene_name=DDX11L1;transcript_type=processed_transcript;transcript_status=KNOWN;transcript_name=DDX11L1-002;exon_number=2;exon_id=ENSE00003582793.1;level=2;transcrip

In [6]:
from Bio import SeqIO

class GffFastaExtractor (object):
    
    ##### FONDAMENTAL METHODS #####
    
    def __init__ (self, seq, gff, out_name="out", feature_types=[]):
        """Init fonction parsing fasta and gff files"""
        
        print("Initialize GffFastaExtractor")
        
        self.seq = seq
        self.gff = gff
        self.out_name = out_name
        self.feature_type = feature_types
        
        # Parse the fasta sequence and store in a dict
        print("  Parsing fasta file")
        with open (self.seq, "r") as seq_in:
            self.seq_dict = SeqIO.to_dict(SeqIO.parse(seq_in, "fasta"))
        
        # Parse the gff file with GffParser
        print("  Parsing gff file")
        self.gff_parser = GffParser (gff_file=gff, feature_types=self.feature_type)

    ##### PUBLIC METHODS #####
            
    def __call__ (self, offset=0, feature_types=[]):
        """Launch the extraction of features """
               
        # Parse the gff parser and the sequence dictionary to extract the sequence of the features
        print("Extract features and write fasta output")
        with open (self.out_name, "w") as seq_out:
            for seq_id, gff_sequence in self.gff_parser.gff_dict.items():
                
                assert seq_id in self.seq_dict, "fasta and gff are incompatible: {} not found in fasta".format(seq_id)
                
                print ("  Extracting features from sequence {}".format(seq_id))
                for feature in gff_sequence.features:
                    
                    if not feature_types or feature.type in feature_types:
                        seq_out.write(">{}\n{}\n".format(
                                str(feature),
                                self.extract_seq(seq_id, feature.start+1, feature.end, feature.strand, offset)))
                
        print ("Done")
        
    ##### PRIVATE METHODS #####
            
    def extract_seq(self, seq_id, start, end, strand, offset=0):
            
        # add ofset if needed and correct if outside of boundaries
        if offset:
            start -= offset
            if start<0:
                start=0
            end += offset
            if end>len(self.seq_dict[seq_id]):
                end=len(self.seq_dict[seq_id])
        
        if strand == "+":
            return str(self.seq_dict[seq_id][start:end].seq)
        
        else:
            return str(self.seq_dict[seq_id][start:end].reverse_complement().seq)
        

## Test without offset restricted to exons 

In [119]:
E = GffFastaExtractor(seq="./test/sample.fa", gff="./test/sample.gff", out_name="./test/test_exons_no_offset.fa")
E()

Initialize GffFastaExtractor
  Parsing fasta file
  Parsing gff file
Extract features and write fasta output
  Extracting features from sequence chr1
  Extracting features from sequence chr2
  Extracting features from sequence chr3
Done


## Test with 50 pb offset restricted to exons 

In [124]:
E = GffFastaExtractor(seq="./test/sample.fa", gff="./test/sample.gff", out_name="./test/test_exons_50pb_offset.fa")
E(offset=50)

Initialize GffFastaExtractor
  Parsing fasta file
  Parsing gff file
Extract features and write fasta output
  Extracting features from sequence chr1
  Extracting features from sequence chr2
  Extracting features from sequence chr3
Done


## Test the with human genome

In [8]:
E = GffFastaExtractor(
    seq="../fasta/GRCh38.primary_assembly.genome.fa",
    gff="../fasta/gencode.v23.primary_assembly.annotation.gff3",
    out_name="../fasta/GRCh38.exon_gencode_v23.fa")

E(offset=100, feature_types=["exon"])

Initialize GffFastaExtractor
  Parsing fasta file
  Parsing gff file
  100000 features parsed
  200000 features parsed
  300000 features parsed
  400000 features parsed
  500000 features parsed
  600000 features parsed
  700000 features parsed
  800000 features parsed
  900000 features parsed
  1000000 features parsed
  1100000 features parsed
  1200000 features parsed
  1300000 features parsed
  1400000 features parsed
  1500000 features parsed
  1600000 features parsed
  1700000 features parsed
  1800000 features parsed
  1900000 features parsed
  2000000 features parsed
  2100000 features parsed
  2200000 features parsed
  2300000 features parsed
  2400000 features parsed
  2500000 features parsed
Extract features and write fasta output
  Extracting features from sequence chr1
  Extracting features from sequence chr2
  Extracting features from sequence chr3
  Extracting features from sequence chr4
  Extracting features from sequence chr5
  Extracting features from sequence chr6
  Ex